<a href="https://colab.research.google.com/github/JohnAle2207/Analisis_sentimientos_tweets/blob/main/Deber_Scrapy_Desarrollo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Instalación de librerias**

In [ ]:
#Instalación
!pip install beautifulsoup4 requests pandas
# Libreria Beautifulsoup se usa para extraer datos de páginas web, parseandolas.
# LIbreria request se utiliza para realizar peticiones http (Get, Post, Etc)
# Libreria pandas se usa para el analisis de datos ya que trabaja con Dataframes.

## **Carga de las dependencias**

In [ ]:
# Load dependencies
import json # modulo para trabajar con archivos JSON
import requests # libreria para hacer peticiones HTTP
import pandas as pd # libreria para procesar datos, proporciona funciones para analíticas
from bs4 import BeautifulSoup as bs # Extraer información de paginas web

## **Funcionamiento**

Conteo de las tablas existentes en la página web

In [ ]:
# Leer el sitio web
r = requests.get('https://es.wikipedia.org/wiki/Anexo:Campeones_de_torneos_de_Grand_Slam_(individual_masculino)')  # Recuperamos el contenido de la web
soup = bs(r.content, 'html.parser') # Parse the HTML content directly

tablas = soup.find_all('table') #Con la dependencia soup podemos contar el número de tablas que tenemos en la página
numero_de_tablas = len(tablas) #Cuenta las veces en que la palabra tabla aparece
print(f"Número de tablas encontradas: {numero_de_tablas}") #Imprime la variable del número de tablas

Número de tablas encontradas: 44


Mostrar los H1 de la página

In [ ]:
h1_todos = soup.find_all('h1') #Buscar todos los H1
for seccion in h1_todos:
    print(seccion.get_text(strip=True))

Anexo:Campeones de torneos de Grand Slam (individual masculino)


Mostrar los H2 de la página

In [ ]:
h2_todos = soup.find_all('h2') #Buscar todos los H1
for seccion in h2_todos:
    print(seccion.get_text(strip=True))

Contenidos
Campeones por año
Máximos ganadores
Otras estadísticas
Véase también
Referencias


Mostrar los archivos src

In [ ]:
src_todos = soup.find_all(src=True)
for elemento in src_todos:
    if elemento['src'].endswith(".png"):
        print(elemento)

<img alt="" aria-hidden="true" class="mw-logo-icon" height="50" src="/static/images/icons/wikipedia.png" width="50"/>
<img alt="" class="mw-file-element" data-file-height="48" data-file-width="48" decoding="async" height="40" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Commons-emblem-question_book_orange.svg/40px-Commons-emblem-question_book_orange.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Commons-emblem-question_book_orange.svg/60px-Commons-emblem-question_book_orange.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Commons-emblem-question_book_orange.svg/80px-Commons-emblem-question_book_orange.svg.png 2x" width="40"/>
<img alt="Bandera del Reino Unido" class="mw-file-element" data-file-height="600" data-file-width="1000" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg/20px-Flag_of_the_United_Kingdom_%283-5%29.svg.png" srcset="//upload.wikimedia.org/

Descargas las imágenes de la página

In [ ]:
url_imgenes = []
for i, imagen in enumerate(src_todos):
    # Verificar si la imagen es un archivo PNG
    if imagen['src'].endswith('png'):
        # Limitar la descarga a solo 10 imágenes
        if i < 10:
            print(imagen['src'])
            r
            # Guardar la imagen en un archivo
            with open(f'imagen_{i}.png', 'wb') as f:
                f.write(r.content)

/static/images/icons/wikipedia.png
//upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Commons-emblem-question_book_orange.svg/40px-Commons-emblem-question_book_orange.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg/20px-Flag_of_the_United_Kingdom_%283-5%29.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg/20px-Flag_of_the_United_Kingdom_%283-5%29.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg/20px-Flag_of_the_United_Kingdom_%283-5%29.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg/20px-Flag_of_the_United_Kingdom_%283-5%29.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg/20px-Flag_of_the_United_Kingdom_%283-5%29.svg.png


Encontrar la tabla y guardar los datos en un csv

In [ ]:
# Busque la tabla (hay que ajustar el selector según la estructura de la página)
table = soup.find('table', class_='plainrowheaders wikitable col1izq')

# Función que convierte una lista de elementos en un dataframe de pandas
def rowsToDataFrame(rows):
    df = pd.DataFrame(rows[1:], columns=rows[0])
    return df

# Actualizar ProcessTableData para encontrar correctamente el cuerpo de la tabla.
def processTableData(tbl):
    rows = []
    # Encuentre el cuerpo de la tabla directamente usando 'tbody'
    for child in tbl.find('tbody').children:  # Itera los items internos de la tabla [<tr>]
        row = []
        for td in child:  # Itera los items internos de la tabla [<td>]
            try:
                item = td.text.replace('\n', '')  # Recupera el texto del elemento [<td>]
                if item:
                    row.append(item)  # Agrega cada elemento de la fila
            except:
                continue
        if len(row) > 0:
            rows.append(row)  # Agrega todos los campos de una fila de la tabla
    return rows

tblRows = processTableData(table) # Ejecuta la función apra porcesar el contenido de la Tabla
df = rowsToDataFrame(tblRows) # Ejecuta la funcion para recupera los datos de la tabla como dataframe de Pandas
df

df.to_csv('cuadro_ganadores.csv')

Guardar los datos en excel

In [ ]:
df.to_excel('cuadro_ganadores1.xlsx')

## **Extraer la página web**

In [ ]:
#Extraer pagina web
!pip install colorama #Instalacion de librerias
import re
from colorama import Fore

web = "https://es.wikipedia.org/wiki/Anexo:Campeones_de_torneos_de_Grand_Slam_(individual_masculino)"
resultado = requests.get(web)
content = resultado.text

Mostrar la página

In [ ]:
print(content)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="es" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Anexo:Campeones de torneos de Grand Slam (individual masculino) - Wikipedia, la enciclopedia libre</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-